#### 一. submit参数调整, 使任务获取更多资源  
```bash
spark-submit 
--class  com.xingyun.test.WordCountCluster 
--num-executors    3            # 配置executor的数量 
--driver-memory    2G         # 配置driver的内存（影响不大）
--executor-memory   2G        # 配置每个executor的内存大小 
--executor-cores   3            # 配置每个executor的cpu core数量 
SparkTest-0.0.1-SNAPSHOT-jar-with-dependencies.jar  
```

1. case1：  
把spark作业提交到Spark Standalone上面。一般自己知道自己的spark测试集群的机器情况。  
举个例子：比如我们的测试集群的机器为每台4G内存，2个CPU core，5台机器。这里以可以申请到最大的资源为例，  
那么  --num-executors  参数就设定为 5，  
那么每个executor平均分配到的资源为：--executor-memory 参数设定为4G，--executor-cores 参数设定为 2 。

2. case2：  
把spark作业提交到Yarn集群上去。那就得去看看要提交的资源队列中大概还有多少资源可以背调度。举个例子：假如可调度的资源配置为：500G内存，100个CPU core，50台机器。 --num-executors  参数就设定为 50，那么每个executor平均分配到的资源为：--executor-memory 参数设定为 10G，--executor-cores 参数设定为 2


#### 二. RDD重用  
1. 对于多次使用的中间RDD, 应该对其缓存, 以避免重复计算  
  1. 直接调用cache()或者presist()方法对指定的RDD进行缓存（持久化）操作
  1. 如下, RDD2应该被缓存
  <img src="img/rdd1.png" height="70%" width="70%">
  2. 缓存级别  
    ```bash
    MEMORY_ONLY       #数据全部缓存在内存中 
    MEMORY_ONLY_2     #数据以双副本的方式缓存在内存中
    MEMORY_ONLY_SER   #数据全部以序列化的方式缓存到内存中
    MEMORY_AND_DISK            #数据一部分缓存在内存中，一部分持久化到磁盘上
    MEMORY_AND_DISK_SER        #数据以序列化的方式一部分缓存在内存中，一部分持久化到磁盘上
    MEMORY_AND_DISK_2          #数据以双副本的方式一部分缓存到内存中，一部分持久化到磁盘上
    DISK_ONLY                  #数据全部持久化到磁盘上
    ```

#### 三. 使用广播变量  
1. Executor内部使用`CachedThreadPool`执行代码, 如果每个线程都使用了Driver机器上的共有变量, 则每个线程都会从从Driver拷贝数据, 从而产生大量网络IO  
2. 使用广播变量后, 每个Executor都会把共有变量从Driver上拷贝到自己的`BlockManager`上, 此后Executor开启的每个线程只要从BlockManager中获取即可, 减少网络IO

#### 四. 使用Kryo序列  
1. Kryo序列化后的对象比java序列化后的对象体积更小  
2. 什么使用使用序列化   
   1. 广播变量进行序列化  
   2. 持久化RDD时使用`Kyro`.  
   3. shuffle阶段

#### 五. 使用`fastutil`序列化Java集合  
1. 当存在共有变量时, 使用广播变量可减小网络传输. 如果共有变量是大型Java集合(List/Map等), 可使用fastutil减小其内存占用.  
 再使用kryo序列化fastutil的集合

#### 六. 优化数据等待时长 
1. Spark的节点运行分配制度  
 Spark在Driver上对Application的每个task任务进行分配之前，都会先计算出每个task要计算的对应的数据分片的位置。Spark的task分配算法优先考虑将task分配到分片数据所在的节点，以此来避免网络间数据传输带来的性能消耗。但是在实际的生产环境, 某些节点的计算资源都已经被占用完了；在这个时候，task会等待一段时间，默认情况是3s，等待时间过了以后，还是没有轮到其执行，task就会去选择一种比较差的本地化级别（要计算的数据与节点的位置关系）去执行。本地化级别可以从spark application的日志中看  
2. spark作业的运行日  
 推荐大家在测试的时候，先用client模式，在本地就直接可以看到比较全的日志。日志里面会显示，starting  task。。。，PROCESSLOCAL、NODE、LOCAL观察大部分task的数据本地化级别，如果大多都是PROCESS_LOCAL，就没有调节的必要。如果是发现，好多的级别都是NODE_LOCAL、ANY，那么最好就去调节一下数据本地化的等待时长调节完，应该是要反复调节，每次调节完以后，再来运行，观察日志看看大部分的task的本地化级别有没有提升；看看，整个spark作业的运行时间有没有缩短。  
3. 参数配置  
spark.locality.wait  
4. 本地化级别  
```bash
PROCESS_LOCAL #进程本地化，代码和数据在同一个进程中，也就是在同一个executor中；计算数据的task由executor执行，数据在executor的BlockManager中；性能最好（图1）
RACK_LOCAL    #机架本地化，数据和task在一个机架的两个节点上；数据需要通过网络在节点之间进行传输（图2）
NODE_LOCAL    #节点本地化，代码和数据在同一个节点中；比如说，数据作为一个HDFSblock块，就在节点上，而task在节点上某个executor中运行；或者是，数据和task在一个节点上的不同executor中；数据需要在进程间进行传输（图3）
ANY           #数据和task可能在集群中的任何地方，而且不在一个机架中，性能最差（图4）
NO_PREF       #对于task来说，数据从哪里获取都一样，没有好坏之分
```

#### 七. spark内存调优  
1. spark的堆内存与堆外内存
spark中的内存溢出分为堆内存和堆外内存.   
堆外内存主要有:用于程序的共享库,Perm Space,线程Stack,一些Memory mapping等,或是类C方式allocate object.    
堆外内存在Spark中可以从逻辑上分成两种:   
  1. 一种是DirectMemory : 在包含disk级别的cache/persist时会用到    
  2. 一种是JVM Overhead(下面统称为off heap): 如JVM永久带   
   这两者在Spark中的大小分别由两个参数设置.  
2. JVM中新生代的eden，survivor1，survivor2的内存占比为8:1:1。  
`minor gc` : 当存活的对象在一个servivor中放不下的时候，就会将这些对象移动到老年代。如果JVM的内存不够大的话，就会频繁的触发`minor gc`，这样会导致一些短生命周期的对象进入到老年代.  
`full gc` : 老年代的对象不断的囤积，最终触发`full gc`。一次`full gc`会使得所有其他程序暂停很长时间。最终严重影响我们的Spark的性能和运行速度。  
3. 堆内存调优  
spark中，堆内存又被划分成了两块儿:  
  1. 给RDD的cache,persist用的数据缓存内存  
  2. 执行算子的工作内存  
spark默认cache和工作内存的比率为6:4. 有时,观察spark作业运行时, 频繁发生gc, 就可能是工作内存太小了, 因此, 可以减少cache堆内存的占比. 使用参数`SparkConf.set("spark.storage.memoryFraction","0.6")`   
4. 内存的调整  
  1. 调大堆外内存:  
  堆外内存溢出, 会导致整个executor失败退出. 当数据量很大时, 到达1亿条, spark作业一运行就会出现类似shuffle file cannot find，executor task lost，out of memory 等这样的错误. 其中shuffle file cannot find，executor task lost都是因为executor退出导致的. 此时可以调大堆外内存` spark.yarn.executor.memoryOverhead=2048`  
  2. 调大连接时长:   
    spark堆内存的永久带gc会触发stop world, 此时, executor和外界的网络链接也会中断.spark默认的网络连接的超时时长是60s；如果卡住60s都无法建立连接的话，那么就宣告失败了。碰到一种情况，有时候报错信息会出现一串类似file id not found，file lost的错误。  
    解决办法为: 在spark-submit脚本中添加参数, 增大executor的连接时长`--conf spark.core.connection.ack.wait.timeout=300`

#### 八. 和shuffle有关的3个参数  
`spark.shuffle.consolidateFiles`，  
`spark.shuffle.file.buffer`，  
`spark.shuffle.memoryFraction`  
1. shuffle的过程  
  1. spark的application会被分为多个stage, 每个stage创建多个task,然后将这些Task分配到各个Executor进程中执行。一个stage的所有Task都执行完毕之后，在各个executor节点上会产生大量的文件，这些文件会通过IO写入磁盘（这些文件存放的时候这个stage计算得到的中间结果），然后Driver就会调度运行下一个stage。下一个stage的Task的输入数据就是上一个stage输出的中间结果。如此循环往复，直到程序执行完毕，最终得到我们想要的结果。  
  2. Spark是根据shuffle类算子来进行stage的划分。如果我们的代码中执行了某个shuffle类算子（比如groupByKey、countByKey、reduceByKey、join等等）每当遇到这种类型的RDD算子的时候，划分出一个stage界限来。
  3. map端缓冲区  
    什么是map端内存缓冲区呢？默认情况下，每个map端的task 输出的一些中间结果在写入磁盘之前，会先被写入到一个临时的内存缓冲区，这个缓冲区的默认大小为32kb，当内存缓冲区满溢之后，才会将产生的中间结果spill到磁盘上。  
  4. reduce端缓冲区   
    reduce端内存占比又是什么呢？reduce端的task在拉取到数据之后，会用一个hashmap的数据结构对各个key对应的value进行汇聚操作。在进行汇聚操作的时候，其使用的内存是由executor进程给分配的，默认将executor的内存的20%分配给reduce task 进行聚合操作使用。这里会有一个问题，当reduce task拉取的数据很多导致其分配的内存放不下的时候，这个时候会将放不下的数据全部spill到磁盘上去。  
    ```bash
    spark.shuffle.file.buffer         #map task的内存缓冲调节参数，默认是32kb
spark.shuffle.memoryFraction      #reduce端聚合内存占比，默认0.2
    ```
2. 调整步骤  
通过监控平台查看每个executor的task的shuffle write和shuffle read的运行次数，如果发现这个指标的运行次数比较多，那么就应该考虑这两个参数的调整了；这个参数调整有一个前提，spark.shuffle.file.buffer参数每次扩大一倍的方式进行调整，spark.shuffle.memoryFraction参数每次增加0.1进行调整。


#### 九 算子调优  
1. 使用mapPartitions算子提高性能  
  1. mapPartition的优点：使用普通的map操作，会对partition中的数据逐条执行.但是使用mapPartitions操作之后，function仅仅会被执行一次，显然性能得到了很大的提升.  
  2. mapPartition的缺点：使用普通的map操作，调用一次function执行一条数据，不会出现内存不够使用的情况；但是使用mapPartitions操作，很显然，如果数据量太过于大的时候，由于内存有限导致发生OOM，内存溢出。  
2. filter操作之后使用coalesce算子提高性能  
  1. filter之后分区内的数据倾斜:  
  经过一次filter操作以后，每个partition的数据量不同程度的变少了，这里就出现了一个问题；由于每个partition的数据量不一样，出现了数据倾斜的问题.  
  2. 解决方案：  
  针对上述出现的问题，我们可以将filter操作之后的数据进行压缩处理；一方面减少partition的数量，从而减少task的数量；另一方面通过压缩处理之后，尽量让每个partition的数据量差不多，减少数据倾斜情况的出现，从而避免某个task运行速度特别慢。coalesce算子就是针对上述出现的问题的一个解决方案  
3. 使用repartition解决SparkSQL低并行度的问题   
在spark项目中，如果在某些地方使用了SparkSQL，那么使用了SparkSQL的那个stage的并行度就没有办法通过手动设置了，而是由程序自己决定。那么，我们通过什么样的手段来提高这些stage的并行度呢？其实解决这个问题的办法就是使partition的数量增多，从而间接的提高了task的并发度，要提高partition的数量，该怎么做呢？就是使用repartition算子，对SparkSQL查询出来的数据重新进行分区操作，此时可以增加分区的个数。